In [1]:
import seaborn as sns
iris=sns.load_dataset('iris')
print(iris.head())

   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa


In [2]:
print(iris.shape)

X=iris.drop('species',axis=1) #axis=1은 열단위, axis=0은 행단위
print(X.shape)

y=iris['species']

(150, 5)
(150, 4)


### 카테고리 실수화

In [3]:
from sklearn.preprocessing import LabelEncoder  #범주형 라벨
import numpy as np

classle=LabelEncoder()
y=classle.fit_transform(iris['species'].values)
print('species labels: ',np.unique(y))  #np.unique() 중복되는 값을 하나로 정리해주는 친구

yo=classle.inverse_transform(y) # 다시 원래대로 문자열로 돌려놓는 친구
print('species: ',np.unique(yo))

species labels:  [0 1 2]
species:  ['setosa' 'versicolor' 'virginica']


### 데이터 분할

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3, random_state=1, stratify=y)  #stratify=y 짱 중요한 옵션

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(105, 4)
(45, 4)
(105,)
(45,)


### (옵션) 입력데이터의 표준화

In [5]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
sc.fit(X_train)
X_train_std=sc.transform(X_train)
X_test_std=sc.transform(X_test)

print(X_train.head())
X_train_std[1:5,]

     sepal_length  sepal_width  petal_length  petal_width
33            5.5          4.2           1.4          0.2
20            5.4          3.4           1.7          0.2
115           6.4          3.2           5.3          2.3
124           6.7          3.3           5.7          2.1
35            5.0          3.2           1.2          0.2


array([[-0.55053619,  0.76918392, -1.16537974, -1.30728421],
       [ 0.65376173,  0.30368356,  0.84243039,  1.44587881],
       [ 1.0150511 ,  0.53643374,  1.0655204 ,  1.18367281],
       [-1.03225536,  0.30368356, -1.44424226, -1.30728421]])

### 모형 추정 및 사례 중심 학습

In [6]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=5, p=2)  #5개의 인접이웃, 거리측정: 유클리드(p=2)

knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [7]:
from sklearn.neighbors import KNeighborsClassifier
knn2=KNeighborsClassifier(n_neighbors=5, p=2)  #5개의 인접이웃, 거리측정: 유클리드(p=2)

knn2.fit(X_train_std,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [8]:
y_train_pred=knn.predict(X_train)
y_test_pred=knn.predict(X_test)

print('Training Samples: %d' %(y_train!=y_train_pred).sum())
print('Test Samples: %d'%(y_test!=y_test_pred).sum())

Training Samples: 2
Test Samples: 1


In [9]:
y_train_pred_std=knn2.predict(X_train_std)
y_test_pred_std=knn2.predict(X_test_std)

print('Training Samples: %d' %(y_train!=y_train_pred_std).sum())
print('Test Samples: %d'%(y_test!=y_test_pred_std).sum())

Training Samples: 4
Test Samples: 3


In [10]:
y_test_pred

array([2, 0, 0, 1, 1, 1, 2, 1, 2, 0, 0, 2, 0, 1, 0, 1, 2, 1, 1, 2, 2, 0,
       1, 2, 1, 1, 1, 2, 0, 2, 0, 0, 1, 1, 2, 2, 0, 0, 0, 1, 2, 2, 1, 0,
       0])

정확도 측정 방법

In [11]:
#accuracy
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_test_pred))

0.9777777777777777


In [12]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_test_pred_std)) #표준화시 성능 저하됨

0.9333333333333333


In [13]:
#confusion matrix
from sklearn.metrics import confusion_matrix
conf=confusion_matrix(y_true=y_test, y_pred=y_test_pred)
print(conf)

[[15  0  0]
 [ 0 15  0]
 [ 0  1 14]]


## KNN 회귀

In [18]:
from sklearn.neighbors import KNeighborsRegressor

regressor=KNeighborsRegressor(n_neighbors=3, weights='distance')  #distance는 가중 평균, defalut=uniform(단순평균)
regressor_uni=KNeighborsRegressor(n_neighbors=3)

training_points=[
  [0.5,0.2,0.1],
  [0.9,0.7,0.3],
  [0.4,0.5,0.7]
]

training_labels=[5.0,6.8,9.0]
regressor.fit(training_points, training_labels)
regressor_uni.fit(training_points, training_labels)

unknown_points=[
  [0.2,0.1,0.7],
  [0.4,0.7,0.6],
  [0.5,0.8,0.1]
]

guesses=regressor.predict(unknown_points)
guesses_uni=regressor_uni.predict(unknown_points)

#데이터가 적어서 세개의 단순회귀 결과가 똑같은듯..?
print('가중회귀: ',guesses)
print('단순회귀: ',guesses_uni)

가중회귀:  [7.28143288 7.76451922 6.8457845 ]
단순회귀:  [6.93333333 6.93333333 6.93333333]
